# Iris classifier
## Imports

In [1]:
import copy
import torch
import numpy as np

from sklearn.model_selection import train_test_split

np.random.seed(99)
torch.manual_seed(99)

## Pre-processing

In [2]:
def iris_to_id(name: str):
    if name == b'Iris-setosa':
        return 0.
    elif name == b'Iris-versicolor':
        return 1.
    elif name == b'Iris-virginica':
        return 2.
    raise Exception(f"Unknown iris {name}")

data = np.loadtxt("iris.csv", delimiter=',', converters={4: iris_to_id})

In [3]:
X = data[:,:4]
y = data[:,4].astype(np.int32)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True, stratify=y)

## Neural Network creation

In [5]:
class MLP(torch.nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super().__init__()
    self.layers = torch.nn.Sequential(
      torch.nn.Linear(input_size, hidden_size, bias=True),
      torch.nn.ReLU(),
      torch.nn.Linear(hidden_size, output_size, bias=True),
    )

  def forward(self, x):
    return self.layers(x)

In [6]:
def train(model, device, train_loader, optimizer, loss_fn=torch.nn.functional.cross_entropy):
    model.train()

    epoch_loss = 0
    n_samples = 0

    for _, (data, target) in enumerate(train_loader):
        # prepare
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()

        # compute
        output = model(data)
        loss = loss_fn(output, target)

        # record
        epoch_loss += loss.item()
        n_samples += output.size(0)

        # adjust
        loss.backward()
        optimizer.step()

    return epoch_loss, n_samples

In [7]:
def test(model, device, test_loader):
    with torch.no_grad():
        model.train(False)
        num_correct = 0
        num_samples = 0

        for _, (x, y) in enumerate(test_loader):
            x = x.to(device)
            y = y.to(device)

            scores = model.forward(x)
            _, y_out = scores.max(1)
            
            num_correct += (y_out == y).sum()
            num_samples += y_out.size(0)
        
        acc = float(num_correct) / float(num_samples)
    return acc

In [8]:
model = MLP(4, 5, 3)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, weight_decay=0.0001)

## Training

In [9]:
device = torch.device('cpu')

train_dataset = torch.utils.data.TensorDataset(torch.Tensor(X_train), torch.LongTensor(y_train))
test_dataset = torch.utils.data.TensorDataset(torch.Tensor(X_test), torch.LongTensor(y_test))

In [10]:
for epoch in range(50):
    train_data = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
    error, num_samples = train(model, device, train_data, optimizer)

In [11]:
test_data = torch.utils.data.DataLoader(test_dataset, batch_size=len(test_dataset))
acc = test(model, device, test_data)
print(f"Test accuracy: {acc * 100:.2f}")

Test accuracy: 100.00


## Pytorch quantization
### Prepare

In [12]:
qmodel_float = copy.deepcopy(model.layers)
qmodel_float.eval()

# fuse layers (weights+activation)
torch.quantization.fuse_modules(qmodel_float, [['0', '1']], inplace=True)

# add quantization of input and output
qmodel_float = torch.nn.Sequential(
    torch.quantization.QuantStub(),
    *qmodel_float,
    torch.quantization.DeQuantStub()
)

In [13]:
# configure quantization
qmodel_float.qconfig = torch.quantization.default_qconfig
qmodel_float=qmodel_float.to('cpu')
qmodel_float.qconfig

QConfig(activation=functools.partial(<class 'torch.ao.quantization.observer.MinMaxObserver'>, quant_min=0, quant_max=127){}, weight=functools.partial(<class 'torch.ao.quantization.observer.MinMaxObserver'>, dtype=torch.qint8, qscheme=torch.per_tensor_symmetric){})

### Calibrate

In [14]:
torch.quantization.prepare(qmodel_float, inplace=True)
with torch.inference_mode():
    for batch_idx, (x, y) in enumerate(test_data):
        x,y = x.to('cpu'), y.to('cpu')
        qmodel_float(x)

### Quantize

In [15]:
qmodel = torch.quantization.convert(qmodel_float, inplace=False)

### Test quantization

In [16]:
# measure accuracy of the quantized model
acc = test(qmodel, 'cpu', test_data)
print(f"Test accuracy: {acc * 100:.2f}")

Test accuracy: 100.00


### Quantization parameters

In [17]:
print(qmodel.state_dict())

OrderedDict([('0.scale', tensor([0.0575])), ('0.zero_point', tensor([0])), ('1.scale', tensor(0.0490)), ('1.zero_point', tensor(0)), ('1._packed_params.dtype', torch.qint8), ('1._packed_params._packed_params', (tensor([[-0.4044,  0.1881,  1.0064,  1.1945],
        [-0.1599,  0.2916, -0.3574, -0.0188],
        [ 0.1787, -0.4985,  1.0064,  0.4420],
        [ 0.0752,  0.8371,  0.2257, -0.5267],
        [ 0.1975,  0.9687, -0.2351, -0.4797]], size=(5, 4), dtype=torch.qint8,
       quantization_scheme=torch.per_tensor_affine, scale=0.009405267424881458,
       zero_point=0), Parameter containing:
tensor([-0.7880, -0.2775, -0.7720,  1.0623,  0.4106], requires_grad=True))), ('3.scale', tensor(0.0932)), ('3.zero_point', tensor(60)), ('3._packed_params.dtype', torch.qint8), ('3._packed_params._packed_params', (tensor([[-0.7590,  0.0143, -0.9166,  0.5012,  0.7017],
        [-0.2220, -0.0072,  0.0931,  0.5012, -0.1719],
        [ 0.8163,  0.0215,  0.4153, -0.5943, -0.3580]], size=(3, 5),
       dt

In [18]:
pre_quantized_params = qmodel_float.state_dict()
for k in pre_quantized_params:
    print(k, pre_quantized_params[k])

0.activation_post_process.eps tensor([1.1921e-07])
0.activation_post_process.min_val tensor(0.2000)
0.activation_post_process.max_val tensor(7.3000)
1.0.weight tensor([[-0.4075,  0.1847,  1.0024,  1.1992],
        [-0.1568,  0.2883, -0.3592, -0.0158],
        [ 0.1752, -0.4949,  1.0089,  0.4388],
        [ 0.0726,  0.8408,  0.2250, -0.5250],
        [ 0.2008,  0.9642, -0.2306, -0.4840]])
1.0.bias tensor([-0.7880, -0.2775, -0.7720,  1.0623,  0.4106])
1.activation_post_process.eps tensor([1.1921e-07])
1.activation_post_process.min_val tensor(0.)
1.activation_post_process.max_val tensor(6.2179)
3.weight tensor([[-0.7603,  0.0155, -0.9130,  0.5034,  0.6988],
        [-0.2238, -0.0065,  0.0930,  0.5039, -0.1733],
        [ 0.8156,  0.0249,  0.4146, -0.5934, -0.3573]])
3.bias tensor([ 0.1456,  0.4571, -0.1649])
3.activation_post_process.eps tensor([1.1921e-07])
3.activation_post_process.min_val tensor(-5.6331)
3.activation_post_process.max_val tensor(6.2031)
